<a href="https://colab.research.google.com/github/rajeevfromkrec/pytorch/blob/master/Pytorch%20embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://nlp.stanford.edu/data/glove.6B.zip


--2020-08-02 13:23:06--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-08-02 13:23:07--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-08-02 13:23:07--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [2]:
!ls

glove.6B.zip  sample_data


In [3]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [6]:
def get_embedding(file):
  f=open(file,'r')
  out=f.read()
  embedding={}
  for line in out:
    out.split()
    word=line[0]
    print(word)
    embedding[word]=npa.arary( [val for val in line[1:]])
  f.close
  return embedding

In [7]:
get_embedding('glove.6B.300d.txt')

t


NameError: ignored

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)


In [9]:
word_to_idx={'hello':0, 'world':1}

In [11]:
embeds=nn.Embedding(2,5)
lookup_tensor= torch.tensor(word_to_idx['hello'], dtype=torch.long)
hello_embeded= embeds(lookup_tensor)

In [12]:
print(hello_embeded)

tensor([ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519],
       grad_fn=<EmbeddingBackward>)


In [ ]:
context_szie=2
embedding_dim=2


In [13]:
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold."""

In [14]:
test_sentence=test_sentence.split()

In [15]:
test_sentence[:10]

['When',
 'forty',
 'winters',
 'shall',
 'besiege',
 'thy',
 'brow,',
 'And',
 'dig',
 'deep']

In [16]:
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [17]:
trigrams1= [ ([test_sentence[i], test_sentence[i+1]] , test_sentence[i+2]  )    for i in range(len(test_sentence)-2)]

In [19]:
trigrams1[:3]

[(['When', 'forty'], 'winters'),
 (['forty', 'winters'], 'shall'),
 (['winters', 'shall'], 'besiege')]

In [20]:
vocab=set(test_sentence)
word_to_idx={word:i for i,word in enumerate(vocab)}

In [25]:
class NgramLangageModel(nn.Module):
  def __init__(self,vocab_size,embed_dim,context_size):
    super().__init__()
    self.embedding=nn.Embedding(vocab_size,embed_dim)
    self.linear1=nn.Linear(embed_dim*context_size,128)
    self.linear2=nn.Linear(128,vocab_size)


  def forward(self, input):
    embeds=self.embedding(input).view((1,-1))
    out=self.linear1(embeds)
    out=F.relu(out)
    out=self.linear2(out)
    log_probs=F.log_softmax(out)
    return log_probs




In [26]:
losses=[]
loss_function=nn.NLLLoss()
model=NgramLangageModel(len(vocab),2,2)

In [27]:
model

NgramLangageModel(
  (embedding): Embedding(97, 2)
  (linear1): Linear(in_features=4, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=97, bias=True)
)

In [28]:
optimizer=optim.SGD(model.parameters(),lr=.001)

In [30]:
for epoch in range(10):
  total_loss=0
  for context,targets in trigrams:
    context_idx= torch.tensor([word_to_idx[w] for w in context],dtype=torch.long)
    model.zero_grad()
    log_probs=model(context_idx)
    loss =loss_function(log_probs, torch.tensor([word_to_idx[targets]],dtype=torch.long))
    loss.backward()
    optimizer.step()

    total_loss  +=loss.item()
  losses.append(total_loss)

print(losses)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  


[503.09426617622375, 500.6617097854614, 498.2647922039032, 495.9034540653229, 493.5766279697418, 491.2841022014618, 489.0254395008087, 486.8005027770996, 484.60845971107483, 482.44896841049194, 480.3221175670624]
